In [1]:
import os
import time

import json
from queue import Queue

import copy
import requests

item_type = 'PSScene'
os.environ['PLANET_API_KEY'] = '80bd00309a7845edb4ee9e0fa62e03b7'
items_collection = {}

def ensure_request(sess, url, wait=.21, exp=2, max_wait=10, max_retry=10):
    retry = 0
    while True:
        response = sess.get(url)
        if response.status_code // 100 == 2:
            break
        time.sleep(wait)
        if wait < max_wait:
            wait *= exp
        retry += 1
        if retry > max_retry:
            print(f'Request failed: {url}')
            print(response)
            print(response.content)
            return False
    return response


def handle_page(page):
    for item in page["features"]:
        try:
            items_collection[item['id']] = copy.deepcopy(item)
        except Exception:
            continue

In [10]:
session = requests.Session()
session.auth = (os.environ['PLANET_API_KEY'], '')

# this large search filter produces all PlanetScope imagery for 1 day
very_large_search = {
    "name": "very_large_search",
    "item_types": ["PSScene"],
    "filter": {
        "type": "AndFilter",
        "config": [
            {
                "type": "DateRangeFilter",
                "field_name": "acquired",
                "config": {
                    "gte": "2021-07-19T12:00:00Z",
                    "lte": "2021-07-20T00:00:00Z"
                }
            },
            {
                "type": "AssetFilter",
                "config": [
                    "ortho_analytic_4b",
                ]
            },
            {
                "type": "StringInFilter",
                "field_name": "satellite_id",
                "config": [
                    "103b",
                ]
            },
            {
                "type":"PermissionFilter",
                "config":[
                    "assets:download"
                ]
            }
        ]
    }
}

In [11]:
saved_search = \
    session.post(
        'https://api.planet.com/data/v1/searches/',
        json=very_large_search)

# after you create a search, save the id. This is what is needed
# to execute the search.
saved_search_id = saved_search.json()["id"]

In [12]:
items_collection={}
def fetch_page(search_url):
    response = ensure_request(session, search_url)
    page = response.json()
    handle_page(page)
    next_url = page["_links"].get("_next")
    if next_url:
        fetch_page(next_url)
first_page = \
    ("https://api.planet.com/data/v1/searches/{}" +
        "/results?_page_size={}").format(saved_search_id, 200)
EOF = False
current_page = first_page
while not EOF:
    response = ensure_request(session, current_page)
    page = response.json()
    print(len(items_collection))
    handle_page(page)
    current_page = page["_links"].get("_next")
    if not current_page:
        EOF = True

0
200
400
600
800
1000
1200
1323


In [13]:
len(items_collection)

1323

In [6]:
item_names=list(items_collection.keys())

In [17]:
fire=['20210719_175940_103b', '20210719_180002_103b', '20210719_175948_103b', '20210719_180001_103b', '20210719_175953_103b', '20210719_175945_103b']
for x in fire:
    assert x in item_names, f'{x} not in item_names'

In [7]:
sas_token='sp=rwdl&st=2023-04-30T13:32:07Z&se=2023-08-22T21:32:07Z&spr=https&sv=2021-12-02&sr=c&sig=YGqrIRcxXdiBsA7Xol77rdCrmtyDNkWSDCamUVBBC2I%3D'
payload = {
            "name": "azure_delivery_order",
            "source_type": "scenes",
            "products": [
                {
                    "item_ids": item_names[:100],
                    "item_type": "PSScene",
                    "product_bundle": "analytic_udm2"
                }
            ],
            "delivery": {
                "azure_blob_storage": {
                    "account": "billtao",
                    "container": "planet",
                    "sas_token": sas_token,
                    # "storage_endpoint_suffix":"",
                    "path_prefix": f"test/"
                }
            }
        }
response = session.post(
        'https://api.planet.com/compute/ops/orders/v2',
        json=payload)
print(response.text)

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/e2dba946-b9c9-4baf-a7fe-8c7f0088e7cc"},"created_on":"2023-04-30T13:33:36.427Z","delivery":{"azure_blob_storage":{"account":"billtao","container":"planet","path_prefix":"test/","sas_token":"<REDACTED>"}},"error_hints":[],"id":"e2dba946-b9c9-4baf-a7fe-8c7f0088e7cc","last_message":"Preparing order","last_modified":"2023-04-30T13:33:36.427Z","name":"azure_delivery_order","products":[{"item_ids":["20210715_122248_0f44","20210715_231609_0f44","20210715_233405_0f44","20210715_140940_0f44","20210715_140851_1_0f44","20210715_170118_1_0f44","20210715_140939_0f44","20210715_233406_0f44","20210715_140853_0f44","20210715_140851_0f44","20210715_140938_0f44","20210715_233248_1_0f44","20210715_233336_0f44","20210715_233357_0f44","20210715_170119_0f44","20210715_140956_1_0f44","20210715_140852_0f44","20210715_170120_0f44","20210715_140937_0f44","20210715_233407_0f44","20210715_170122_0f44","20210715_231610_0f44","20210715_233356_0f44","20

In [19]:
sas_token='sp=rwdl&st=2023-04-30T13:32:07Z&se=2023-08-22T21:32:07Z&spr=https&sv=2021-12-02&sr=c&sig=YGqrIRcxXdiBsA7Xol77rdCrmtyDNkWSDCamUVBBC2I%3D'
# Cut the item into chunks of 500
items_collection_chunks = [list(items_collection.keys())[i:i + 500] for i in range(0, len(items_collection), 500)]
# Submit the chunks to the API
for (i, chunk) in enumerate(items_collection_chunks):
    payload = {
            "name": f"103b_one_day_{i}",
            "products": [
                {
                    "item_ids": chunk,
                    "item_type": "PSScene",
                    "product_bundle": "analytic_udm2"
                }
            ],
            "delivery": {
                "azure_blob_storage": {
                    "account": "billtao",
                    "container": "planet",
                    "sas_token": sas_token,
                    # "storage_endpoint_suffix":"",
                    "path_prefix": f"1048_whole_day/"
                }
            }
        }
    response = session.post(
        'https://api.planet.com/compute/ops/orders/v2',
        json=payload)
    print(response.text)

{"_links":{"_self":"https://api.planet.com/compute/ops/orders/v2/bd98fae3-5f0c-4168-b5fc-187c04dd47ed"},"created_on":"2023-05-03T20:50:07.193Z","delivery":{"azure_blob_storage":{"account":"billtao","container":"planet","path_prefix":"1048_whole_day/","sas_token":"<REDACTED>"}},"error_hints":[],"id":"bd98fae3-5f0c-4168-b5fc-187c04dd47ed","last_message":"Preparing order","last_modified":"2023-05-03T20:50:07.193Z","name":"103b_one_day_0","products":[{"item_ids":["20210719_162208_103b","20210719_162209_103b","20210719_162210_103b","20210719_175607_103b","20210719_145536_103b","20210719_175608_103b","20210719_175721_103b","20210719_175609_103b","20210719_175610_103b","20210719_175621_103b","20210719_175611_103b","20210719_162207_103b","20210719_162211_103b","20210719_175612_103b","20210719_175622_103b","20210719_175613_103b","20210719_175623_103b","20210719_175653_103b","20210719_162212_103b","20210719_175614_103b","20210719_175624_103b","20210719_162213_103b","20210719_175555_103b","202107